**Test global constraint:** A grid is defined at the start of the notebook, and it is verified that the `global_flag` is set to **0** only when row‐uniqueness, column‐uniqueness, and cell‐validity all pass, and to **1** if any one of these constraints fails.  


In [1]:
import sys, os
sys.path.append(os.path.dirname(os.getcwd()))

from oracle.oracle import oracle
from utils.indexer import Indexer
from utils.grid import pretty_print_grids

from math import ceil, log2
from qiskit import QuantumRegister, QuantumCircuit, transpile
from qiskit_aer import AerSimulator


In [2]:
# Define your grid here
grid = [
    [0, 1, 2],   
    [2, 0, 1],
    [1, 2, 0]  
]

# Derive dimensions automatically
n = len(grid)
m = len(grid[0])

# Print the initial grid
flat_init = tuple(x for row in grid for x in row)
print("Initial grid:")
pretty_print_grids({flat_init: 1}, n, m)


Initial grid:
Count = 1
+---+---+---+
| 0 | 1 | 2 |
+---+---+---+
| 2 | 0 | 1 |
+---+---+---+
| 1 | 2 | 0 |
+---+---+---+



In [3]:
k = max(1, int(ceil(log2(max(n, m)))))
num_anc = k + 2*m + n - 1
idx = Indexer(grid, num_anc)

qr = QuantumRegister(idx.total_qubits, name="q")
qc = QuantumCircuit(qr, name="row_uniqueness_demo")

# Load the pre-filled grid into data qubits
idx.initialize_grid(qc)

# Apply the column uniqueness oracle and measurement
oracle(qc, qr, idx)
qc.measure_all()

In [4]:
# Simulate and collect counts
sim = AerSimulator(method="matrix_product_state")
tcirc = transpile(qc, sim, optimization_level=3)
result = sim.run(tcirc, shots=1).result()
counts = result.get_counts()

print("Total qubits:", qc.num_qubits)
print("Raw counts:", counts)

Total qubits: 32
Raw counts: {'00000000000000001001010010100100': 1}


In [5]:
# Extract the bitstring and reconstruct the grid
bitstr = next(iter(counts))
bits   = list(map(int, bitstr[::-1]))  # reverse so index matches qubit

# Print the role and values of each measured qubit
print("Measured qubits:")
for q in range(idx.total_qubits):
    role = idx.pretty(q)
    print(f" q[{q:2d}] = {bits[q]}   ⟶ {role}")

# Reconstruct the n*m grid from data region
recovered = [[0]*m for _ in range(n)]
for i in range(n):
    for j in range(m):
        v = 0
        for b in range(k):
            v |= (bits[idx.data(i, j, b)] << b)
        recovered[i][j] = v

flat_rec = tuple(x for row in recovered for x in row)
print("\nRecovered grid after oracle + measurement:")
pretty_print_grids({flat_rec: counts[bitstr]}, n, m)

# Check the global flag and report validity
gf = idx.global_flag()
flag_val = bits[gf]
print(f"global_flag (q[{gf}]) = {flag_val}")

if flag_val == 0:
    print("\nThe grid is valid")
else:
    print("\nThe grid is invalid")


Measured qubits:
 q[ 0] = 0   ⟶ data(0,0,0)
 q[ 1] = 0   ⟶ data(0,0,1)
 q[ 2] = 1   ⟶ data(0,1,0)
 q[ 3] = 0   ⟶ data(0,1,1)
 q[ 4] = 0   ⟶ data(0,2,0)
 q[ 5] = 1   ⟶ data(0,2,1)
 q[ 6] = 0   ⟶ data(1,0,0)
 q[ 7] = 1   ⟶ data(1,0,1)
 q[ 8] = 0   ⟶ data(1,1,0)
 q[ 9] = 0   ⟶ data(1,1,1)
 q[10] = 1   ⟶ data(1,2,0)
 q[11] = 0   ⟶ data(1,2,1)
 q[12] = 1   ⟶ data(2,0,0)
 q[13] = 0   ⟶ data(2,0,1)
 q[14] = 0   ⟶ data(2,1,0)
 q[15] = 1   ⟶ data(2,1,1)
 q[16] = 0   ⟶ data(2,2,0)
 q[17] = 0   ⟶ data(2,2,1)
 q[18] = 0   ⟶ row_flag
 q[19] = 0   ⟶ col_flag
 q[20] = 0   ⟶ cell_valid_flag
 q[21] = 0   ⟶ global_flag
 q[22] = 0   ⟶ ancilla(0)
 q[23] = 0   ⟶ ancilla(1)
 q[24] = 0   ⟶ ancilla(2)
 q[25] = 0   ⟶ ancilla(3)
 q[26] = 0   ⟶ ancilla(4)
 q[27] = 0   ⟶ ancilla(5)
 q[28] = 0   ⟶ ancilla(6)
 q[29] = 0   ⟶ ancilla(7)
 q[30] = 0   ⟶ ancilla(8)
 q[31] = 0   ⟶ ancilla(9)

Recovered grid after oracle + measurement:
Count = 1
+---+---+---+
| 0 | 1 | 2 |
+---+---+---+
| 2 | 0 | 1 |
+---+---+---+
| 1 | 2